In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import pandas_datareader as pdr

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 백테스트 날짜, 초기 잔고, 리밸런싱 주기
start = datetime(2000, 1, 1)
end = datetime(2020, 12, 31)
init_balance = 1000

rebalancing_months = [12, 3] # 리밸런싱 실행 월
rebalancing_target = -1 # -1 월 말, 0 월 초
rebalancing_dates = [0, 1, 2] # 월말 3영업일, 월초 2영업일
# rebalancing_months = [12 , 3, 10] # 리밸런싱 실행 월
# rebalancing_target = -1 # -1 월 말, 0 월 초
# rebalancing_dates = [-3, -2, -1, 0, 1] # 월말 3영업일, 월초 2영업일

fee = 0.5 # 거래비용 및 슬리피지. %(퍼센트) 단위

portfolio = []
# 포트1: 주식, 채권, 원자재 6/3/1 국산
portfolio.append({
        'asset': ['251350.KS', '195980.KS', '304660.KS', '305080.KS', '132030.KS'],
        'ratio': [42, 18, 18, 12, 10],
        'name': 'Portfolio 1'
})
# 포트2: 주식, 채권, 원자재 6/3/1 미국산
portfolio.append({
        'asset': ['URTH', 'EEM', 'TLT', 'IEF', 'IAU'],
        'ratio': [42, 18, 18, 12, 10],
        'name': 'Portfolio 2'
})
# 벤치마크: S&P500
portfolio.append({
        'asset': ['^GSPC'],
        'ratio': [1],
        'name': 'S&P500'
})

In [3]:
def individual_port_price(port): # 개별 포트폴리오의 가격 리스트에대한 df 생성
    
    # 각 종목 별 수정 주가 데이터를 크롤링 해 리스트로 시계열 데이터 시리즈를 값으로 갖는 리스트 생성
    price_list = [pdr.get_data_yahoo(asset, start, end)['Adj Close'] for asset in port['asset']]

    df_list = pd.concat(price_list, axis=1) #주가 데이터 리스트를 데이터프레임으로 변환
    df_list.columns = port['asset'] # 포트를 구성하는 종목 명을 컬럼 명으로
    first_day = df_list.dropna().index.min() # 전 종목 결측치 없는 최초 일자를 거래시작일로 지정
    df_price = df_list[first_day:].fillna(method='pad') # 최초일 이후의 결측치는 전일종가로 채움
    end_day = df_price.index.max()
        
    return first_day, end_day, df_price

In [4]:
def individual_port_balance(port): # 개별 포트에 대한 잔고 계산
    first_day, end_day, df_price = individual_port_price(port)
    
    df_price['Date'] = df_price.index
    df_price['month'] = df_price.index.month
    df_price['year_month'] = df_price.index.strftime('%Y-%m')
    
    df_target = df_price.loc[df_price["month"].isin(rebalancing_months)].groupby('year_month').apply(lambda x: x.iloc[rebalancing_target])
    
    rebal_list = list(df_target['Date'])
    if rebal_list[0] != first_day:
        rebal_list.insert(0, first_day)
    if rebal_list[-1] != end_day:
        rebal_list.append(end_day)
    
    df_ratio = df_price.drop(['year_month', 'month', 'Date'], axis=1)
    
    tmp_balance = init_balance
    new_col = [f'{asset}_@' for asset in port['asset']]
    for i in range(len(rebal_list)-1):
        tmp = df_ratio.loc[(df_ratio.index >= rebal_list[i]) & (df_ratio.index <= rebal_list[i+1])]
        for asset, ratio in zip(port['asset'], port['ratio']):
            changing_val = tmp[asset] / tmp[asset].iloc[0] * ratio/sum(port['ratio']) * tmp_balance
            df_ratio.loc[(df_ratio.index >= rebal_list[i]) & (df_ratio.index <= rebal_list[i+1]), f'{asset}_@'] \
            = changing_val
        tmp_balance = df_ratio.loc[rebal_list[i+1], new_col].sum()
    #     print(tmp_balance)
    #     print(df_ratio.loc[rebal_list[i+1]])
    #     print('================================')
    df_ratio['Balance'] = df_ratio[new_col].sum(axis=1)
        
    return df_ratio['Balance']

In [5]:
def merge_balance(balance_list): # 각각 포트 별 잔액 합치기

    port_bal = pd.concat(balance_list, axis=1)
    port_bal.columns = [portfolio[i]['name'] for i in range(len(portfolio))]
    port_bal = port_bal.dropna()

    for port in port_bal.columns:
        port_bal[port] = port_bal[port] / float(port_bal[port].head(1)) * init_balance

    return port_bal

In [6]:
def performance_indicator(etf): # etf는 df["etf명"]
    date = etf.index.tolist()
    balance = etf.values.tolist()
    ###########최종 평가액, 첫거래일, 마지막거래일, 테스트기간###########
    final_balance = balance[-1]
    first_day, last_day = date[0], date[-1]
    time_period = ((last_day - first_day).days)/365
    
    CAGR = ((balance[-1]/balance[0]) ** (1 / float(time_period)) - 1) * 100
    cum_return = ((balance[-1]/balance[0] - 1) * 100)
    ###########신고가, MDD, 일 변동률###############################
    historic_high = 0
    historic_high_percent = 0
    MDD = 0
    drawdown_list = [0]
    daily_change_list = [0]
    
    for i in range(1, len(date)):     
        if historic_high < balance[i]:
            historic_high = balance[i] 

        drawdown_list.append((balance[i]/historic_high - 1) * 100)
        
        if MDD > drawdown_list[i]:
            MDD = drawdown_list[i]
        
        daily_change_list.append((balance[i]/balance[i-1] - 1)*100)
        
    historic_high_percent = (historic_high/balance[0] - 1)*100
    ###########STDEV, 샤프지수신고가################################
    # 일단위 변동성 연율화 하기 위해 252일 영업일 기준으로 루트 252 곱 함
    STDEV = np.std(daily_change_list) * np.sqrt(252) 
    sharpe_ratio = np.mean(daily_change_list) / np.std(daily_change_list) * np.sqrt(252) 
    
    my_str = []
    result_list = [init_balance, final_balance, CAGR, MDD, STDEV, sharpe_ratio]
    for my_number in result_list:
        if my_number in [CAGR, MDD]:
            my_str.append(str(round(my_number, 2))+' %')
        elif my_number in [final_balance]:
            my_str.append(str(round(my_number)))
        else : 
            my_str.append(str(round(my_number, 2)))
    
    first, last = first_day.strftime('%Y-%m-%d'), last_day.strftime('%Y-%m-%d')
    print(f'테스트기간 : {first} ~ {last} / {round(time_period,1)} 년')
    
    return my_str, drawdown_list

In [7]:
bal_list = [individual_port_balance(port) for port in portfolio] #모든 포트의 잔액를 리스트로

KeyError: 'Date'

In [ ]:
port_bal = merge_balance(bal_list)
port_bal.head(3)

In [ ]:
tmp = {
        'index': [],
        'data': [],
        'drawdown': []
    }
########### 포트폴리오 리턴 결과 표 ###############################
port_list = [portfolio[i]['name'] for i in range(len(portfolio))]
for port in port_list:
    tmp['index'].append(port)
    tmp['data'].append(performance_indicator(port_bal[port])[0])
    tmp['drawdown'].append(performance_indicator(port_bal[port])[1])

portfolio_return = pd.DataFrame(data=tmp['data'], index=tmp['index'], \
                   columns=['Init Balance', 'Final Balance', 'CAGR', 'MDD', 'STDEV', 'Sharpe Ratio'])

########### drawdown 리스트 -> 데이터프레임 변환 ###################
tmp_drawdown = [pd.DataFrame(data=tmp['drawdown'][i], index=port_bal.index, columns=[portfolio[i]['name']]) for i in range(len(tmp['drawdown']))]
df_drawdown = pd.concat(tmp_drawdown, axis=1)

########### Annual Returns / Monthly Returns ###################
port_bal['year'] = port_bal.index.strftime('%Y')
port_bal['year_month'] = port_bal.index.strftime('%Y-%m')
annual = port_bal.groupby('year').apply(lambda x: x.iloc[[-1]]).reset_index('Date').drop(['year_month', 'year', 'Date'], axis=1)
monthly = port_bal.groupby('year_month').apply(lambda x: x.iloc[[-1]]).reset_index('Date').drop(['year_month', 'year', 'Date'], axis=1)

for port in port_list:
    for i in (range(len(annual)-1, 0, -1)):
        annual[port][i] = round((annual[port][i] / annual[port][i-1] - 1), 4) * 100
    for i in (range(len(monthly)-1, 0, -1)):
        monthly[port][i] = round((monthly[port][i] / monthly[port][i-1] - 1), 4) * 100
annual.iloc[0] = (annual.iloc[0] / init_balance - 1) * 100
monthly.iloc[0] = (monthly.iloc[0] / init_balance - 1) * 100

In [ ]:
portfolio_return

In [ ]:
# 그래픽 크기 설정
plt.rcParams["font.size"] = 16
plt.rcParams["figure.figsize"] = (20, 10)

port_bal.plot()
df_drawdown.plot()
annual.plot.bar()
monthly.plot.bar()
plt.grid()
plt.legend()
plt.show()


In [ ]:
portfolio_return.to_excel('portfolio_return_4_input_end.xlsx')